In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random
import warnings
warnings.filterwarnings('ignore')

import matplotlib.image as img

#Import Necessary Libraries
from os import listdir
from pickle import dump, load
import pickle 

from tqdm.notebook import tqdm

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import L1L2

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add, concatenate

In [74]:
image_directory='../../../data/explore/image_caption_generator/Image_captioning_Dataset/Flicker8k_Dataset/'
img_caption_pkl=pd.read_pickle('../../../data/explore/image_caption_generator/set_0.pkl')

In [75]:

img_names, img_captionIds, img_captions=[], [], []
for row in img_caption_pkl:
    cnt_=sum([1  if char =='#' else 0 for char in row ])
    if cnt_>1:
        print(row, cnt_)
    row=row.split('#')
    img_name, img_caption=row[0], row[1]
    captionId, caption = img_caption.split('\t')
    img_names.append(img_name.split('.jpg')[0])
    img_captionIds.append(captionId)
    img_captions.append(caption)

image_captions=pd.DataFrame({'image':img_names, 'captionId':img_captionIds, 'caption':img_captions})
print(image_captions.shape)
image_captions.head(2)

2837799692_2f1c50722a.jpg#4	Closeup of football player # 25 . 2
3273625566_2454f1556b.jpg#4	A race dog have a muzzle and be wear striped jersey # 8 . 2
2837799692_2f1c50722a.jpg#3	a university of Miami football player # 25 . 2
3147913471_322ea231d9.jpg#2	Florida man 's basketball player # 33 shoot basketball . 2
2833582518_074bef3ed6.jpg#4	A # 2 greyhound dog be run around a track . 2
2769605231_dae8b30201.jpg#1	A black greyhound , with a green and white jersey ( # 7 ) be run on a track . 2
(25000, 3)


,image,captionId,caption
0,318667317_108c402140,4,A man in a black hoodie be hold a paper sign .
1,2072574835_febf0c5fb9,4,Three race dog be run out of the start gate on...


In [76]:
image_captions[image_captions['image']=='319847657_2c40e14113'].head()

,image,captionId,caption
9948,319847657_2c40e14113,3,A woman be tug on a white sheet and laugh
16427,319847657_2c40e14113,1,A girl in a purple shirt be have pull someone ...


In [77]:
image_captions.drop_duplicates().shape

(25000, 3)

In [43]:
image_captions.captionId.nunique(), image_captions.image.nunique(), image_captions.caption.nunique()

(5, 8037, 24817)

In [44]:
# # image_directory='Image_captioning_Dataset/Flicker8k_Dataset'
# for image in random.sample(list(image_captions.image), 3):
#     plt.figure()
#     print(image_captions[image_captions['image']==image].caption.iloc[0])
#     im = img.imread(image_directory + '/'+ image + '.jpg')
#     plt.imshow(im) 
#     plt.show()


In [15]:

def image_feature_extraction(image_directory, model):
    '''
        extract image features from the given directory using pretrained model
    '''
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    # summarize
    print(model.summary())
    # extract features from each photo
    features = dict()
    for name in tqdm(listdir(image_directory)):
        # load an image from file
        filename = image_directory + '/' + name
        image = load_img(filename, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the given model
        image = preprocess_input(image)
        # get features
        feature = model.predict(image, verbose=0)
        # get image id
        image_id = name.split('.')[0]
        # store feature
        features[image_id] = feature
#         print('>%s' % name)
    return features

In [16]:
# # model=ResNet50()
# resnet50=load_model('../../../data/explore/image_caption_generator/resnet50.h5')
# image_features = image_feature_extraction(image_directory, resnet50)
# print('Extracted Features: %d' % len(image_features))
# # save to file
# dump(image_features, open('../../../data/explore/image_caption_generator/output/image_features_reset50.pkl', 'wb'))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________


Extracted Features: 8091


In [78]:
# load features from pickle
with open(os.path.join('../../../data/explore/image_caption_generator/output/image_features_reset50.pkl'), 'rb') as f:
    image_features = load(f)

In [91]:
len(image_features), image_captions.image.nunique()

(8091, 8036)

In [92]:
image_captions=image_captions[image_captions['image'].isin(image_features.keys())]
print(len(image_features), image_captions.image.nunique(), image_captions.shape)

8091 8036 (24996, 3)


In [93]:
def caption_preprocessing(caption):
    # preprocessing steps
    # convert to lowercase, delete digits, special chars & additional spaces
    caption = caption.lower().replace('[^A-Za-z]', '').replace('\s+', ' ')
    # add start and end tags to the caption
    caption = 'startseq ' + " ".join([word for word in caption.split() if len(word)>1]) + ' endseq'
    return caption
image_captions['caption']=image_captions['caption'].apply(caption_preprocessing)
image_captions.head()

,image,captionId,caption
0,318667317_108c402140,4,startseq startseq man in black hoodie be hold ...
1,2072574835_febf0c5fb9,4,startseq startseq three race dog be run out of...
2,3083016677_5782bc337c,4,startseq startseq two motorcycle with two ride...
3,95734038_2ab5783da7,0,startseq startseq man on bike nest to river en...
4,241346146_f27759296d,1,startseq startseq football player try tackle a...


In [94]:
image_captions_all=image_captions['caption'].values
print(len(image_captions_all))
image_captions_all[0:5]

24996


array(['startseq startseq man in black hoodie be hold paper sign endseq endseq',
       'startseq startseq three race dog be run out of the start gate on track endseq endseq',
       'startseq startseq two motorcycle with two rider each endseq endseq',
       'startseq startseq man on bike nest to river endseq endseq',
       'startseq startseq football player try tackle another player who have the ball endseq endseq'],
      dtype=object)

In [95]:
# tokenize the text
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(image_captions_all)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

5340


In [96]:
# get maximum length of the caption available
max_len_caption = max([len(caption.split()) for caption in image_captions_all])
max_len_caption

33

In [97]:
image_names = image_captions['image'].unique()
index = int(len(image_names) * 0.80)
image_names_train=image_names[:index]
image_names_test=image_names[index:]
image_captions_train = image_captions[image_captions['image'].isin(image_names_train)]
image_captions_test = image_captions[image_captions['image'].isin(image_names_test)]
print(image_captions_train.shape, image_captions_test.shape)

(21000, 3) (3996, 3)


In [ ]:
# create data generator to get data in batch (avoids session crash)
def data_generator(image_captions, image_features, tokenizer, max_length, vocab_size, batch_size):
    # loop over images
    X1, X2, y = list(), list(), list()
    n = 0
#     while True:
    for image in image_captions.image.unique():
        n += 1
        captions = image_captions[image_captions['image']==image].caption.unique()
        # process each caption
        for caption in captions:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([caption])[0]
            # split the sequence into X, y pairs
            for i in range(1, len(seq)):
                # split into input and output pairs
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                # store the sequences
                X1.append(image_features[image][0])
                X2.append(in_seq)
                y.append(out_seq)
    return [X1, X2], y
#             if n == batch_size:
#                 X1, X2, y = np.array(X1), np.array(X2), np.array(y)
#                 yield [X1, X2], y
#                 X1, X2, y = list(), list(), list()
#                 n = 0

In [98]:
# create data generator to get data in batch (avoids session crash)
def data_generator(image_captions, image_features, tokenizer, max_length, vocab_size, batch_size):
    # loop over images
    X1, X2, y = list(), list(), list()
    n = 0
    while True:
        for image in image_captions.image.unique():
            n += 1
            captions = image_captions[image_captions['image']==image].caption.unique()
            # process each caption
            for caption in captions:
                # encode the sequence
                seq = tokenizer.texts_to_sequences([caption])[0]
                # split the sequence into X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pairs
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    
                    # store the sequences
                    X1.append(image_features[image][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n == batch_size:
                X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                yield [X1, X2], y
                X1, X2, y = list(), list(), list()
                n = 0

In [103]:
from tensorflow.keras.models import Model
# encoder model
# image feature layers
img_input = Input(shape=(2048,))
img_feature_layer1 = Dropout(0.4)(img_input)
img_feature_layer2 = Dense(128, activation='relu')(img_feature_layer1)
# sequence feature layers
seq_input = Input(shape=(max_len_caption,))
seq_embedding = Embedding(vocab_size, 256, mask_zero=True)(seq_input)
seq_dr_layer = Dropout(0.4)(seq_embedding)
seq_layer1 = LSTM(64, return_sequences=True, bias_regularizer=L1L2(l1=.01, l2=0))(seq_dr_layer)
# seq_layer2 = LSTM(32, return_sequences=True, bias_regularizer=L1L2(l1=.01, l2=0))(seq_layer1)
# seq_layer3 = LSTM(64, return_sequences=True, bias_regularizer=L1L2(l1=.01, l2=0))(seq_layer2)
# seq_layer4 = LSTM(64, return_sequences=True, bias_regularizer=L1L2(l1=.01, l2=0))(seq_layer3)
seq_layer5 = LSTM(128, bias_regularizer=L1L2(l1=.01, l2=0))(seq_layer1)

# decoder model
decoder1 = add([img_feature_layer2, seq_layer5])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[img_input, seq_input], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# plot the model
# plot_model(model, show_shapes=True)

In [104]:
# model.summary()

In [ ]:
epochs = 2
batch_size = 32
steps = len(image_names_train) // batch_size

# create train data generator
generator_train = data_generator(image_captions_train, image_features, tokenizer, max_len_caption, vocab_size, batch_size)
# create train data generator
generator_validation = data_generator(image_captions_test, image_features, tokenizer, max_len_caption, vocab_size, batch_size)
# fit for one epoch
model_history=model.fit(generator_train, epochs=5, steps_per_epoch=steps)
    

# save model
# model.save('model_' + str(i) + '.h5')

Epoch 1/5
200/200 [==============================] - 73s 364ms/step - loss: 6.3630
Epoch 2/5
191/200 [===========================>..] - ETA: 3s - loss: 5.0322

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [50]:
len(image_features['318667317_108c402140.jpg'][0])

1000